Sveučilište u Zagrebu  
Fakultet elektrotehnike i računarstva  
Mentor: Krešimir Križanović

### Klasifikacija_bolesti_na_temelju_podataka_o_ekspresiji_gena
## SEMINAR 1 2024./2025.  
Domagoj Sviličić

http://www.fer.unizg.hr

Učitavanje potrebnih biblioteka

In [2]:
# Učitaj osnovne biblioteke...
import sklearn
import matplotlib.pyplot as plt
# %pylab inline
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [3]:
def plot_2d_clf_problem(X, y, h=None):
    '''
    Plots a two-dimensional labeled dataset (X,y) and, if function h(x) is given, 
    the decision surfaces.
    '''
    assert X.shape[1] == 2, "Dataset is not two-dimensional"
    if h!=None : 
        # Create a mesh to plot in
        r = 0.04  # mesh resolution
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, r),
                             np.arange(y_min, y_max, r))
        XX=np.c_[xx.ravel(), yy.ravel()]
        try:
            Z_test = h(XX)
            if Z_test.shape == ():
                # h returns a scalar when applied to a matrix; map explicitly
                Z = np.array(list(map(h,XX)))
            else :
                Z = Z_test
        except ValueError:
            # can't apply to a matrix; map explicitly
            Z = np.array(list(map(h,XX)))
        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        plt.contourf(xx, yy, Z, cmap=plt.cm.Pastel1)

    # Plot the dataset
    plt.scatter(X[:,0],X[:,1], c=y, cmap=plt.cm.tab20b, marker='o', s=50);
    plt.show()

### 2. Višeklasna (5 klasa) klasifikacija karcinoma dojke logističkom regresijom

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Učitavanje podataka iz CSV datoteke
file_path = 'Breast_GSE45827.csv'  # zamijeniti s točnom putanjom do datoteke
data = pd.read_csv(file_path)

# Brojanje ukupnog broja stupaca u podacima
total_columns = data.shape[1]
print(f"Ukupan broj stupaca: {total_columns}")

# Izdvajanje oznaka klase i značajki
y = data.iloc[:, 1]  # drugi stupac s oznakama klase
X = data.iloc[:, 2:]  # svi stupci nakon drugog s razinama ekspresije gena

# Provjera dimenzija
print(f"Dimenzije značajki (X): {X.shape}")
print(f"Dimenzije oznaka (y): {y.shape}")

# Dodavanje stupca jedinica u matricu dizajna X
X = np.c_[np.ones(X.shape[0]), X]  # dodavanje prvog stupca jedinica  //provjeriti na konzultacijama da li je to uopće potrebno raditi

# Provjera dimenzija nakon dodavanja stupca jedinica
print(f"Dimenzije značajki nakon dodavanja stupca jedinica (X): {X.shape}")

# Funkcija za treniranje modela i evaluaciju točnosti za dani omjer podjele
def evaluate_model(X, y, test_size):
    # Podjela podataka
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)  # postavljanje random_state za replikaciju rezultata
    # Treniranje modela logističke regresije
    model = LogisticRegression(max_iter=1000, solver='lbfgs')  # automatski odabir metode za višeklasnu/multinomijalnu klasifikaciju
    model.fit(X_train, y_train)
    # Predikcija i točnost
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Evaluacija za različite omjere
results = {
    "7:3": evaluate_model(X, y, test_size=0.3),
    "6:4": evaluate_model(X, y, test_size=0.4),
    "1:1": evaluate_model(X, y, test_size=0.5),
}

# Ispis rezultata
print("Točnost za različite omjere podjele podataka:")
for ratio, accuracy in results.items():
    print(f"Omjer {ratio}: Točnost = {accuracy:.4f}")



Ukupan broj stupaca: 54677
Dimenzije značajki (X): (151, 54675)
Dimenzije oznaka (y): (151,)
Dimenzije značajki nakon dodavanja stupca jedinica (X): (151, 54676)
Točnost za različite omjere podjele podataka:
Omjer 7:3: Točnost = 0.8913
Omjer 6:4: Točnost = 0.9016
Omjer 1:1: Točnost = 0.9342
